In [ ]:
from curl_cffi import requests
import pandas as pd
import time
import random
from datetime import datetime

# ==========================================
# CONFIGURAÇÕES
# ==========================================
SEASON_ID = 72034
TOURNAMENT_ID = 325
# Headers para simular um navegador real
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Referer": "https://www.sofascore.com/"
}

def get_matches_ids(session):
    """
    Busca todas as partidas de todas as rodadas (1 a 38)
    Retorna uma lista de dicionários com metadados básicos da partida.
    """
    matches_metadata = []
    url_round = "https://api.sofascore.com/api/v1/unique-tournament/{}/season/{}/events/round/{}"
    
    print("--- Coletando IDs das partidas (Rodadas 1-38) ---")
    
    for rodada in range(1, 38):
        try:
            r = session.get(url_round.format(TOURNAMENT_ID, SEASON_ID, rodada), impersonate="chrome")
            if r.status_code == 200:
                events = r.json().get('events', [])
                for event in events:
                    # Só processa se a partida já tiver terminado ou tiver cobertura
                    if event.get('status', {}).get('type') == 'finished':
                        matches_metadata.append({
                            'match_id': event.get('id'),
                            'rodada': rodada,
                            'data': datetime.fromtimestamp(event.get('startTimestamp')),
                            'time_casa': event.get('homeTeam', {}).get('name'),
                            'time_visitante': event.get('awayTeam', {}).get('name'),
                            'placar_casa': event.get('homeScore', {}).get('display'),
                            'placar_visitante': event.get('awayScore', {}).get('display')
                        })
            else:
                print(f"Erro ao ler rodada {rodada}: Status {r.status_code}")
            
            # Pausa curta entre rodadas
            time.sleep(0.5)
            
        except Exception as e:
            print(f"Exceção na rodada {rodada}: {e}")
            
    print(f"Total de partidas encontradas: {len(matches_metadata)}")
    return matches_metadata

def process_player_stats(player_entry, team_name, match_meta):
    """
    Extrai e achata os dados de um jogador específico.
    """
    player_info = player_entry.get('player', {})
    stats = player_entry.get('statistics', {})
    
    # Dados base (Metadados da partida + Info do Jogador)
    row = {
        # Metadados da Partida
        'match_id': match_meta['match_id'],
        'rodada': match_meta['rodada'],
        'data': match_meta['data'],
        'confronto': f"{match_meta['time_casa']} vs {match_meta['time_visitante']}",
        'time_jogador': team_name,
        
        # Info do Jogador
        'player_id': player_info.get('id'),
        'player_name': player_info.get('name'),
        'player_slug': player_info.get('slug'),
        'position': player_entry.get('position'),
        'shirt_number': player_entry.get('shirtNumber'),
        'substitute': player_entry.get('substitute', False),
        'captain': player_entry.get('captain', False),
    }
    
    # Adiciona as estatísticas dinamicamente (flatten)
    # O Python 3.9+ permite unir dicionários com | ou update
    row.update(stats)
    
    # Remove dicionários aninhados que possam vir dentro de stats (ex: ratingVersions)
    # para não quebrar o DataFrame
    if 'ratingVersions' in row:
        del row['ratingVersions']
    if 'statisticsType' in row:
        del row['statisticsType']
        
    return row

# ==========================================
# EXECUÇÃO PRINCIPAL
# ==========================================

all_players_data = []

with requests.Session() as session:
    # 1. Obter lista de jogos
    matches = get_matches_ids(session)
    
    # 2. Iterar sobre cada jogo para pegar lineups (A parte demorada)
    print("\n--- Coletando Estatísticas de Jogadores (Lineups) ---")
    
    total_matches = len(matches)
    
    for i, match in enumerate(matches):
        match_id = match['match_id']
        url_lineups = f"https://api.sofascore.com/api/v1/event/{match_id}/lineups"
        
        try:
            r = session.get(url_lineups, impersonate="chrome")
            
            if r.status_code == 200:
                data = r.json()
                
                # Processar Time da Casa
                if 'home' in data and 'players' in data['home']:
                    for p in data['home']['players']:
                        row = process_player_stats(p, match['time_casa'], match)
                        all_players_data.append(row)
                
                # Processar Time Visitante
                if 'away' in data and 'players' in data['away']:
                    for p in data['away']['players']:
                        row = process_player_stats(p, match['time_visitante'], match)
                        all_players_data.append(row)
                        
            elif r.status_code == 404:
                print(f"Lineups não disponíveis para jogo ID {match_id}")
            else:
                print(f"Erro request jogo {match_id}: {r.status_code}")

        except Exception as e:
            print(f"Erro processamento jogo {match_id}: {e}")
            
        # LOG DE PROGRESSO
        if (i + 1) % 10 == 0:
            print(f"Processado {i + 1}/{total_matches} jogos...")
        
        # IMPORTANTE: Delay aleatório para evitar bloqueio (Rate Limit)
        # Dorme entre 0.8 e 1.5 segundos por jogo
        time.sleep(random.uniform(0.8, 1.5))

# ==========================================
# CRIAÇÃO DO DATAFRAME
# ==========================================

if all_players_data:
    df_players = pd.DataFrame(all_players_data)
    
    # Preenche NaNs com 0 (para estatísticas que o jogador não teve no jogo)
    # Apenas nas colunas numéricas de estatísticas
    # Identificamos colunas de metadados para não preencher com 0 (ex: posição, nome)
    cols_metadata = ['match_id', 'rodada', 'data', 'confronto', 'time_jogador', 
                     'player_id', 'player_name', 'player_slug', 'position', 
                     'shirt_number', 'substitute', 'captain']
    
    # Pega todas as colunas que NÃO são metadados (ou seja, são stats)
    cols_stats = [c for c in df_players.columns if c not in cols_metadata]
    
    # Preenche vazios com 0 nas colunas de stats
    df_players[cols_stats] = df_players[cols_stats].fillna(0)
    
    print("\n--- Concluído ---")
    print(f"Total de registros (linhas): {len(df_players)}")
    print(f"Total de colunas (estatísticas encontradas): {len(df_players.columns)}")
    
    # Exemplo de visualização
    print(df_players.head())
    
    # Opcional: Salvar
    # df_players.to_csv("sofascore_players_full.csv", index=False)
else:
    print("Nenhum dado de jogador coletado.")


ANÁLISE DE DESEMPENHO - RODADA 1
Total de Atletas que Jogaram: 293

PONTUAÇÃO LUCCA

1. Jogadores acima de X pontos (por Posição):
         >2  >4  >6  >8  >10  >12  >14  >16
Posição                                    
G        14   9   4   3    2    0    0    0
D        66  52  29  17   10    6    3    0
M        83  55  35  19   10    5    1    0
F        29  20  13  10    5    2    2    2

2. Distribuição de Posições nos Melhores Rankings:
         Top 5  Top 10  Top 15  Top 20  Top 25  Top 30  Top 35  Top 40
Posição                                                               
G            0       0       0       0       2       2       2       3
D            2       5       6       8       9      13      14      15
M            1       3       7       8      10      10      12      14
F            2       2       2       4       4       5       7       8

3. Top 10 Melhores da Rodada 1:
            Nome Posição                Time  PONTUACAO_LUCCA_MATCH
   Pablo Vegetti       F 

In [10]:
from curl_cffi import requests
import pandas as pd
import time
import random

# ==========================================
# CONFIGURAÇÃO
# ==========================================
URL_TEMPLATE = "https://www.sofascore.com/api/v1/fantasy/round/770/players?page={}"
PAGINA_INICIAL = 0
PAGINA_FINAL = 29

dados_coletados = []

print(f"--- Iniciando coleta de Fantasy (Páginas {PAGINA_INICIAL} a {PAGINA_FINAL}) ---")

# Session mantém a conexão TCP aberta, acelerando o processo
with requests.Session() as session:
    for page in range(PAGINA_INICIAL, PAGINA_FINAL + 1):
        try:
            url = URL_TEMPLATE.format(page)
            
            # impersonate="chrome" é crucial para evitar erro 403
            response = session.get(url, impersonate="chrome")
            
            if response.status_code == 200:
                data = response.json()
                players_list = data.get('players', [])
                
                # Se a lista vier vazia, encerra o loop antes do final para não perder tempo
                if not players_list:
                    print(f"Página {page} vazia. Encerrando coleta.")
                    break

                # Extração dos dados solicitados
                for item in players_list:
                    # Objetos aninhados
                    fantasy_player = item.get('fantasyPlayer', {}) # As vezes vem direto, as vezes dentro de fantasyPlayer
                    
                    # A estrutura do JSON que você mandou mostra "players" -> lista de objetos.
                    # Mas no JSON de exemplo havia uma chave "fantasyPlayer" e outra direta.
                    # Vou fazer uma verificação defensiva para pegar os dados onde quer que estejam.
                    
                    # Se o item tiver 'player' direto, usa ele. Se tiver 'fantasyPlayer', entra um nível.
                    if 'player' in item:
                        root = item
                    elif 'fantasyPlayer' in item:
                        root = item['fantasyPlayer']
                    else:
                        continue # Pula se não identificar estrutura

                    player_info = root.get('player', {})
                    team_info = root.get('team', {})
                    
                    dados_coletados.append({
                        'Nome': player_info.get('name'),
                        'Posição': player_info.get('position'),
                        'Camisa': player_info.get('jerseyNumber'),
                        'ID': player_info.get('id'),
                        'Time': team_info.get('name'),
                        'Preço': root.get('price'),
                        'Nota_SofaScore': root.get('averageRating')
                    })
                
                print(f"Página {page} ok - {len(players_list)} atletas coletados.")
            
            else:
                print(f"Erro na página {page}: Status {response.status_code}")

            # Delay aleatório para parecer comportamento humano
            time.sleep(random.uniform(0.5, 1.2))

        except Exception as e:
            print(f"Exceção na página {page}: {e}")

# ==========================================
# CRIAÇÃO DO DATAFRAME
# ==========================================

if dados_coletados:
    df_fantasy = pd.DataFrame(dados_coletados)
    
    # Tratamento de tipos numéricos e preenchimento de nulos
    df_fantasy['Preço'] = pd.to_numeric(df_fantasy['Preço'], errors='coerce').fillna(0)
    df_fantasy['Nota_SofaScore'] = pd.to_numeric(df_fantasy['Nota_SofaScore'], errors='coerce').fillna(0)
    df_fantasy['ID'] = df_fantasy['ID'].fillna(0).astype(int)
    
    print("\n" + "="*50)
    print(f"COLETA FINALIZADA: {len(df_fantasy)} Jogadores")
    print("="*50)
    
    # Exibe os top 5 mais caros para conferência
    print(df_fantasy.sort_values(by='Preço', ascending=False).head())
else:
    print("Nenhum dado foi coletado.")

--- Iniciando coleta de Fantasy (Páginas 0 a 29) ---
Página 0 ok - 30 atletas coletados.
Página 1 ok - 30 atletas coletados.
Página 2 ok - 30 atletas coletados.
Página 3 ok - 30 atletas coletados.
Página 4 ok - 30 atletas coletados.
Página 5 ok - 30 atletas coletados.
Página 6 ok - 30 atletas coletados.
Página 7 ok - 30 atletas coletados.
Página 8 ok - 30 atletas coletados.
Página 9 ok - 30 atletas coletados.
Página 10 ok - 30 atletas coletados.
Página 11 ok - 30 atletas coletados.
Página 12 ok - 30 atletas coletados.
Página 13 ok - 30 atletas coletados.
Página 14 ok - 30 atletas coletados.
Página 15 ok - 30 atletas coletados.
Página 16 ok - 30 atletas coletados.
Página 17 ok - 30 atletas coletados.
Página 18 ok - 30 atletas coletados.
Página 19 ok - 30 atletas coletados.
Página 20 ok - 30 atletas coletados.
Página 21 ok - 30 atletas coletados.
Página 22 ok - 30 atletas coletados.
Página 23 ok - 30 atletas coletados.
Página 24 ok - 30 atletas coletados.
Página 25 ok - 30 atletas coleta

In [153]:
import pandas as pd
import numpy as np

# ==============================================================================
# CONFIGURAÇÃO DA ANÁLISE
# ==============================================================================
RODADA_ALVO = 23  # <--- ALTERE AQUI PARA A RODADA QUE DESEJA ANALISAR

# ==============================================================================
# 1. PREPARAÇÃO DA BASE DE ESTATÍSTICAS (df_players)
# ==============================================================================

df = df_players.copy()

# A. Preenchimento de Nulos
cols_stats = [
    'rating', 'ownGoals', 'yellowCards', 'redCards', 'totalOffside', 
    'dispossessed', 'minutesPlayed', 'penaltySave', 'penaltyWon', 
    'penaltyConceded', 'penaltyMiss', 'totalPass', 'accuratePass', 
    'totalLongBalls', 'accurateLongBalls', 'duelWon', 'duelLost', 
    'wonContest', 'totalContest', 'keyPass', 'wasFouled', 'fouls',
    'totalClearance', 'outfielderBlock', 'interceptionWon', 'wonTackle', 
    'savedShotsFromInsideTheBox', 'saves', 'punches', 'goodHighClaim', 
    'accurateKeeperSweeper', 'goals', 'goalAssist', 'goalLineClearance',
    'shotOffTarget', 'onTargetScoringAttempt', 'hitWoodwork', 'goalsPrevented'
]

for col in cols_stats:
    if col not in df.columns:
        df[col] = 0.0
    else:
        df[col] = df[col].fillna(0)

# B. Cálculo de Placar e Gols Sofridos
temp_teams = df['confronto'].str.split(' vs ', expand=True)
df['home_team_name'] = temp_teams[0]
df['away_team_name'] = temp_teams[1]
df['is_home'] = df['time_jogador'] == df['home_team_name']

goals_home = df[df['is_home']].groupby('match_id')['goals'].sum()
og_away    = df[~df['is_home']].groupby('match_id')['ownGoals'].sum()
goals_away = df[~df['is_home']].groupby('match_id')['goals'].sum()
og_home    = df[df['is_home']].groupby('match_id')['ownGoals'].sum()

match_scores = pd.DataFrame({
    'score_home': goals_home.add(og_away, fill_value=0),
    'score_away': goals_away.add(og_home, fill_value=0)
})

df = df.merge(match_scores, on='match_id', how='left').fillna(0)
df['gols_sofridos_partida'] = np.where(df['is_home'], df['score_away'], df['score_home'])

# ==============================================================================
# 2. UNIÃO COM A BASE DO FANTASY
# ==============================================================================

if 'df_fantasy' in locals():
    # Mantém a rodada na base de stats, mas traz info cadastral do fantasy
    cols_fantasy = ['ID', 'Nome', 'Posição', 'Time', 'Preço', 'Nota_SofaScore']
    fantasy_clean = df_fantasy[cols_fantasy].drop_duplicates(subset='ID')
    
    # Inner join para garantir dados oficiais
    df_merged = df.merge(fantasy_clean, left_on='player_id', right_on='ID', how='inner')
else:
    raise ValueError("ERRO: df_fantasy não foi carregado na memória.")

# ==============================================================================
# 3. CÁLCULO DE PONTUAÇÃO (LUCCA E SOFA)
# ==============================================================================

# --- LUCCA ---
cond_nota = [(df_merged['rating']>=9), (df_merged['rating']>=8), (df_merged['rating']>=7), (df_merged['rating']>=6.5), (df_merged['rating']>=6), (df_merged['rating']>=3)]
val_nota = [3, 2, 1, 0, -1, -2]
df_merged['L_nota'] = np.select(cond_nota, val_nota, default=0)

df_merged['L_negativos'] = (
    (df_merged['ownGoals'] * -2) + (df_merged['yellowCards'] * -1) + (df_merged['totalOffside'] * -0.25) +
    (df_merged['dispossessed'] * -0.25) + (df_merged['penaltyConceded'] * -2) + 
    (df_merged['penaltyMiss'] * -3) + (df_merged['fouls'] * -0.5)
)
df_merged['L_red'] = np.where(df_merged['redCards'] > 0, -3, 0)
df_merged['L_part'] = np.where(df_merged['minutesPlayed'] > 75, 1, 0)

p_passe = np.where(df_merged['totalPass']>0, df_merged['accuratePass']/df_merged['totalPass'], 0)
p_long = np.where(df_merged['totalLongBalls']>0, df_merged['accurateLongBalls']/df_merged['totalLongBalls'], 0)
p_duel = np.where((df_merged['duelWon']+df_merged['duelLost'])>0, df_merged['duelWon']/(df_merged['duelWon']+df_merged['duelLost']), 0)
p_drib = np.where(df_merged['totalContest']>0, df_merged['wonContest']/df_merged['totalContest'], 0)

df_merged['L_bonus'] = (
    np.where((df_merged['totalPass']>=40) & (p_passe>=0.90), 1, 0) +
    np.where((df_merged['accurateLongBalls']>=3) & (p_long>=0.60), 1, 0) +
    np.where((df_merged['duelWon']>=3) & (p_duel>=0.50), 1, 0) +
    np.where((df_merged['wonContest']>=3) & (p_drib>=0.60), 1, 0)
)

real_shot = (df_merged['onTargetScoringAttempt'] - df_merged['hitWoodwork'] - df_merged['goals']).clip(lower=0)
df_merged['L_acoes'] = (
    (df_merged['keyPass'] * 0.75) + (df_merged['penaltySave'] * 5) + (df_merged['penaltyWon'] * 2) +
    (df_merged['wasFouled'] * 0.5) + (df_merged['shotOffTarget'] * 0.75) + 
    (real_shot * 1.5) + (df_merged['hitWoodwork'] * 3)
)

df_merged['L_def'] = np.where(df_merged['Posição'] != 'G', 
    (df_merged['totalClearance']*0.1) + (df_merged['outfielderBlock']*0.25) + 
    (df_merged['interceptionWon']*0.5) + (df_merged['wonTackle']*0.75) + (df_merged['goalLineClearance']*2), 0
)

saves_out = (df_merged['saves'] - df_merged['savedShotsFromInsideTheBox']).clip(lower=0)
df_merged['L_gk'] = np.where(df_merged['Posição'] == 'G',
    (df_merged['savedShotsFromInsideTheBox']*1.0) + (saves_out*0.5) + 
    (df_merged['accurateKeeperSweeper']*1) + (df_merged['goalLineClearance']*2)+
    (df_merged['goalsPrevented']*2), 0
)

pos_L_G = {'G':6,'D':6,'M':6,'F':6}
pos_L_A = {'G':4,'D':4,'M':4,'F':4}
pos_L_SG = {'G':4,'D':3,'M':0,'F':0}

df_merged['L_pos'] = (
    df_merged.apply(lambda x: x['goals']*pos_L_G.get(x['Posição'],4) + x['goalAssist']*pos_L_A.get(x['Posição'],3), axis=1) +
    df_merged.apply(lambda x: pos_L_SG.get(x['Posição'],0) if (x['gols_sofridos_partida']==0 and x['minutesPlayed'] > 0) else 0, axis=1) +
    np.where(df_merged['Posição'].isin(['G','D']), df_merged['gols_sofridos_partida']*-0.5, 0)
)

df_merged['PONTUACAO_LUCCA_MATCH'] = df_merged[['L_nota','L_negativos','L_red','L_part','L_bonus','L_acoes','L_def','L_gk','L_pos']].sum(axis=1)

# --- SOFA ---
df_merged['S_nota'] = df_merged['L_nota']
df_merged['S_negativos'] = (
    (df_merged['ownGoals']*-2) + (df_merged['yellowCards']*-1) + ((df_merged['totalOffside']//2)*-1) +
    ((df_merged['dispossessed']//3)*-1) + (df_merged['penaltyConceded']*-2) + (df_merged['penaltyMiss']*-3)
)
df_merged['S_red'] = df_merged['L_red']
df_merged['S_part'] = np.where(df_merged['minutesPlayed'] > 60, 2, np.where(df_merged['minutesPlayed'] > 0, 1, 0))
df_merged['S_bonus'] = df_merged['L_bonus']
df_merged['S_acoes'] = ((df_merged['keyPass']//2)*1) + ((df_merged['wasFouled']//3)*1) + (df_merged['penaltySave']*5) + (df_merged['penaltyWon']*2)

df_merged['S_def'] = np.where(df_merged['Posição'] != 'G',
    ((df_merged['totalClearance']//5)*1) + ((df_merged['blockedScoringAttempt']//2)*1) + 
    ((df_merged['interceptionWon']//3)*1) + ((df_merged['wonTackle']//3)*1), 0
)

df_merged['S_gk'] = np.where(df_merged['Posição'] == 'G',
    ((df_merged['savedShotsFromInsideTheBox']//2)*1) + ((saves_out//3)*1) + 
    (((df_merged['punches']+df_merged['goodHighClaim'])//2)*1) + (df_merged['accurateKeeperSweeper']*1) + 
    (df_merged['goalLineClearance']*2), 0
)

pos_S_G = {'G':6,'D':6,'M':5,'F':4}
pos_S_A = {'G':4,'D':4,'M':3,'F':3}
pos_S_SG = {'G':4,'D':4,'M':0,'F':0}

df_merged['S_pos'] = (
    df_merged.apply(lambda x: x['goals']*pos_S_G.get(x['Posição'],4) + x['goalAssist']*pos_S_A.get(x['Posição'],3), axis=1) +
    df_merged.apply(lambda x: pos_S_SG.get(x['Posição'],0) if (x['gols_sofridos_partida']==0 and x['minutesPlayed']>=60) else 0, axis=1) +
    np.where(df_merged['Posição'].isin(['G','D']), ((df_merged['gols_sofridos_partida']//2)*-1), 0)
)

df_merged['PONTUACAO_SOFA_MATCH'] = df_merged[['S_nota','S_negativos','S_red','S_part','S_bonus','S_acoes','S_def','S_gk','S_pos']].sum(axis=1)

# Arredondamento
def arredondar_personalizado(series):
    vals = series.round(2)
    decimals = (vals * 100).round().astype(int) % 100
    last_digit = decimals % 10
    adjustment = np.where((last_digit == 6) | (last_digit == 9), 0.01, 0.0)
    return (vals + adjustment).round(2)

df_merged['PONTUACAO_LUCCA_MATCH'] = arredondar_personalizado(df_merged['PONTUACAO_LUCCA_MATCH'])
df_merged['PONTUACAO_SOFA_MATCH']  = arredondar_personalizado(df_merged['PONTUACAO_SOFA_MATCH'])

# ==============================================================================
# 4. FILTRO E ANÁLISE POR RODADA
# ==============================================================================

# Filtra pela rodada definida E jogadores que jogaram (>0 minutos)
df_rodada = df_merged[
    (df_merged['rodada'] == RODADA_ALVO) & 
    (df_merged['minutesPlayed'] > 0)
].copy()

if df_rodada.empty:
    print(f"\n--- AVISO: Nenhum dado encontrado para a Rodada {RODADA_ALVO} ou rodada ainda não iniciou ---")
else:
    print(f"\nANÁLISE DE DESEMPENHO - RODADA {RODADA_ALVO}")
    print(f"Total de Atletas que Jogaram: {len(df_rodada)}")

    def analise_faixas_pontuacao(df_input, col_score):
        """Conta jogadores por posição acima de cada corte de pontuação"""
        cortes = [-2, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14]
        data = {}
        
        for corte in cortes:
            # Filtra quem fez > corte
            mask = df_input[col_score] > corte
            # Conta por posição (G, D, M, F)
            contagem = df_input[mask]['Posição'].value_counts()
            data[f'>{corte}'] = contagem
            
        resumo = pd.DataFrame(data).fillna(0).astype(int)
        # Garante ordem das linhas
        ordem = ['G', 'D', 'M', 'F']
        return resumo.reindex([p for p in ordem if p in resumo.index])

    def analise_distribuicao_top(df_input, col_score):
        """Analisa a posição dos Top N pontuadores"""
        tops = [3, 5, 10, 20, 40, 80]
        data = {}
        
        # Ordena do maior para o menor
        df_sorted = df_input.sort_values(by=col_score, ascending=False)
        
        for n in tops:
            # Pega os top N
            top_n = df_sorted.head(n)
            # Conta posições
            contagem = top_n['Posição'].value_counts()
            data[f'Top {n}'] = contagem
            
        resumo = pd.DataFrame(data).fillna(0).astype(int)
        ordem = ['G', 'D', 'M', 'F']
        return resumo.reindex([p for p in ordem if p in resumo.index])

    # --- RELATÓRIO LUCCA ---
    print("\n" + "="*70)
    print("PONTUAÇÃO LUCCA")
    print("="*70)
    
    print("\n1. Jogadores acima de X pontos (por Posição):")
    print(analise_faixas_pontuacao(df_rodada, 'PONTUACAO_LUCCA_MATCH'))
    
    print("\n2. Distribuição de Posições nos Melhores Rankings:")
    print(analise_distribuicao_top(df_rodada, 'PONTUACAO_LUCCA_MATCH'))
    
    print(f"\n3. Top 10 Melhores da Rodada {RODADA_ALVO}:")
    cols_top = ['Nome', 'Posição', 'Time', 'PONTUACAO_LUCCA_MATCH']
    print(df_rodada.nlargest(50, 'PONTUACAO_LUCCA_MATCH')[cols_top].to_string(index=False))


ANÁLISE DE DESEMPENHO - RODADA 23
Total de Atletas que Jogaram: 313

PONTUAÇÃO LUCCA

1. Jogadores acima de X pontos (por Posição):
         >-2   >0  >1  >2  >3  >4  >5  >6  >7  >8  >9  >10  >12  >14
Posição                                                             
G         22   19  18  16  15  13   9   8   7   4   3    3    2    1
D         96   86  79  66  57  50  43  33  28  23  15   13    9    5
M        132  112  99  79  68  57  51  41  29  21  16   11    6    3
F         59   47  39  30  27  21  19  13  12   7   6    4    3    2

2. Distribuição de Posições nos Melhores Rankings:
         Top 3  Top 5  Top 10  Top 20  Top 40  Top 80
Posição                                              
G            0      1       1       2       3       7
D            1      1       4       9      15      29
M            1      2       3       6      16      31
F            1      1       2       3       6      13

3. Top 10 Melhores da Rodada 23:
                  Nome Posição             

In [154]:
import pandas as pd
import numpy as np

# ==============================================================================
# 1. AGRUPAMENTO ACUMULADO (CAMPEONATO INTEIRO)
# ==============================================================================

print("\nPROCESSANDO DADOS ACUMULADOS DO CAMPEONATO...")

# Cria flag de jogo real (minutos > 0)
df_merged['jogou_real'] = np.where(df_merged['minutesPlayed'] > 0, 1, 0)

# Agrupa por dados do Fantasy (ID e Nome fixos)
df_final = df_merged.groupby(['ID', 'Nome', 'Posição', 'Time', 'Preço']).agg({
    'PONTUACAO_LUCCA_MATCH': 'sum',
    'PONTUACAO_SOFA_MATCH': 'sum',
    'jogou_real': 'sum', # Soma apenas partidas que entrou em campo
    'goals': 'sum',
    'goalAssist': 'sum'
}).reset_index()

# Renomeia
df_final = df_final.rename(columns={
    'PONTUACAO_LUCCA_MATCH': 'Total_Lucca',
    'PONTUACAO_SOFA_MATCH': 'Total_Sofa',
    'jogou_real': 'Jogos'
})

# Calcula Médias
# Evita divisão por zero
df_final['Media_Lucca'] = np.where(df_final['Jogos'] > 0, (df_final['Total_Lucca'] / df_final['Jogos']).round(2), 0)
df_final['Media_Sofa'] = np.where(df_final['Jogos'] > 0, (df_final['Total_Sofa'] / df_final['Jogos']).round(2), 0)

# ==============================================================================
# 2. FILTRO: APENAS JOGADORES COM > 7 JOGOS
# ==============================================================================

df_validos = df_final[df_final['Jogos'] > 5].copy()

print(f"Total de Atletas Analisados (> 7 jogos): {len(df_validos)}")

# ==============================================================================
# 3. FUNÇÕES DE ANÁLISE DE MÉDIA
# ==============================================================================

def analise_faixas_media(df_input, col_media):
    """Conta jogadores por posição acima de cada corte de MÉDIA"""
    # Cortes ajustados para médias (valores típicos de fantasy)
    cortes = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]
    data = {}
    
    for corte in cortes:
        mask = df_input[col_media] >= corte
        contagem = df_input[mask]['Posição'].value_counts()
        data[f'Média >={corte}'] = contagem
        
    resumo = pd.DataFrame(data).fillna(0).astype(int)
    ordem = ['G', 'D', 'M', 'F']
    return resumo.reindex([p for p in ordem if p in resumo.index])

def analise_distribuicao_top_media(df_input, col_media):
    """Analisa a posição dos Top N melhores em MÉDIA"""
    tops = [5, 10, 15, 20, 25, 30, 35, 40, 50, 100]
    data = {}
    
    # Ordena pela média
    df_sorted = df_input.sort_values(by=col_media, ascending=False)
    
    for n in tops:
        top_n = df_sorted.head(n)
        contagem = top_n['Posição'].value_counts()
        data[f'Top {n}'] = contagem
        
    resumo = pd.DataFrame(data).fillna(0).astype(int)
    ordem = ['G', 'D', 'M', 'F']
    return resumo.reindex([p for p in ordem if p in resumo.index])

def analise_faixas_media_perc(df_input, col_media):
    """
    Conta % de jogadores por posição acima de cada corte de MÉDIA.
    Retorna a porcentagem relativa ao total de jogadores daquela posição.
    """
    cortes = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]
    data = {}
    
    # 1. Calcula o total de jogadores por posição na base filtrada (Denominador)
    total_por_posicao = df_input['Posição'].value_counts()
    
    for corte in cortes:
        # Filtra quem tem média >= corte
        mask = df_input[col_media] >= corte
        contagem = df_input[mask]['Posição'].value_counts()
        
        # 2. Calcula % relativa ao total daquela posição
        # O Pandas alinha automaticamente os índices (G com G, D com D, etc.)
        percentual = (contagem / total_por_posicao * 100).fillna(0)
        data[f'Média >={corte}'] = percentual
        
    # Cria DataFrame, arredonda e formata
    resumo = pd.DataFrame(data).fillna(0).round(1)
    
    # Adiciona o símbolo de % para facilitar a leitura
    resumo = resumo.applymap(lambda x: f"{x}%")
    
    # Ordena as linhas
    ordem = ['G', 'D', 'M', 'F']
    return resumo.reindex([p for p in ordem if p in resumo.index])

# ==============================================================================
# 4. EXIBIÇÃO DOS RESULTADOS
# ==============================================================================

cols_show = ['Nome', 'Posição', 'Time', 'Jogos', 'Preço', 'Total_Lucca', 'Media_Lucca']

# --- ANÁLISE LUCCA ---
print("\n" + "="*70)
print("ANÁLISE DO CAMPEONATO - PONTUAÇÃO LUCCA (MÉDIA)")
print("="*70)

print("\n1. Quantidade de Jogadores com Média Superior a X:")
print(analise_faixas_media(df_validos, 'Media_Lucca'))

print("\n1. Porc. de Jogadores com Média Superior a X:")
print(analise_faixas_media_perc(df_validos, 'Media_Lucca'))

print("\n2. Distribuição de Posições nos Melhores Rankings de Média:")
print(analise_distribuicao_top_media(df_validos, 'Media_Lucca'))

print(f"\n3. Top 20 Melhores do Campeonato (Média Lucca):")
print(df_validos.nlargest(240, 'Media_Lucca')[cols_show].to_string(index=False))


PROCESSANDO DADOS ACUMULADOS DO CAMPEONATO...
Total de Atletas Analisados (> 7 jogos): 545

ANÁLISE DO CAMPEONATO - PONTUAÇÃO LUCCA (MÉDIA)

1. Quantidade de Jogadores com Média Superior a X:
         Média >=0.0  Média >=1.0  Média >=2.0  Média >=3.0  Média >=4.0  \
Posição                                                                    
G                 28           27           26           26           22   
D                172          166          147          128           94   
M                241          225          191          140           84   
F                 99           93           68           52           33   

         Média >=5.0  Média >=6.0  Média >=7.0  Média >=8.0  Média >=9.0  
Posição                                                                   
G                 15            6            2            1            1  
D                 58           29            9            1            1  
M                 52           21           12    

C:\Users\lucca\AppData\Local\Temp\ipykernel_8792\228235965.py:103: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  resumo = resumo.applymap(lambda x: f"{x}%")
